In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from paths import mkl_data
import requests # API library

import random
import time
import os

from isoweek import Week

import requests # API library

import numpy as np
import pandas as pd
import json
import re

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam
from pyro.infer import Predictive

import torch

# Pyro-specific imports
import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam, ClippedAdam


In [ ]:
#Model
def simple_model(obs=None):
    skill1 = pyro.sample("skill1", dist.Normal(0., 1.))
    skill2 = pyro.sample("skill2", dist.Normal(0., 1.))

    sigma1 = 1; sigma2 = 2
    perf1 = pyro.sample("perf1", dist.Normal(skill1, sigma1))
    perf2 = pyro.sample("perf2", dist.Normal(skill2, sigma2))

    with pyro.plate("data", len(obs)):
        y = pyro.sample("obs", dist.Categorical(), obs=obs)
    
    return y

In [ ]:
#Make guide and optimizer
auto_guide = pyro.infer.autoguide.AutoNormal(simple_model)
adam = pyro.optim.Adam({"lr": 0.02})
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(simple_model, auto_guide, adam, elbo)

In [ ]:
#Clear param store 
pyro.clear_param_store()

# These should be reset each training loop.
model = simple_model
auto_guide = pyro.infer.autoguide.AutoNormal(model)
adam = pyro.optim.Adam({"lr": 0.02})  # Consider decreasing learning rate.
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)


# Do gradient steps
n_steps = 1000
X_torch = torch.tensor(df_matches['team1_win'].values, dtype=torch.int32)
for step in range(n_steps):
    elbo = svi.step(X_torch)
    if step % 100 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))